# Retry parser

While in some cases it is possible to fix any parsing mistakes by only looking at the output, in other cases it isn't. An example of this is when the output is not just in the incorrect format, but is partially complete. Consider the below example.

In [2]:
from langchain.output_parsers import (
    OutputFixingParser,
    PydanticOutputParser,
)
from langchain.prompts import (
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAI

In [3]:
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

In [4]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n Do not forget use double quotes",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [5]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [6]:
bad_response = "{'action': 'search'}"

If we try to parse this response as is, we will get an error:

In [7]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {'action': 'search'}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

If we try to use the `OutputFixingParser` to fix this error, it will be confused - namely, it doesn't know what to actually put for action input.

In [8]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [9]:
fix_parser.parse(bad_response)

Action(action='search', action_input='input')

Instead, we can use the RetryOutputParser, which passes in the prompt (as well as the original output) to try again to get a better response.

In [10]:
from langchain.output_parsers import RetryWithErrorOutputParser

In [11]:
retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=OpenAI(temperature=0)
)

In [12]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='leo di caprios gf')